In [1]:

import Thymio as th
import time
import videopath1 as vp
import numpy as np
import threading
import vision.detect_aruco as da
import queue
import cv2
def draw_path(img, path):
    for i in range(len(path)-1):
        cv2.line(img, path[i], path[i+1], (0, 255, 0), 2)
    return img

In [ ]:

channel_pos = queue.Queue(maxsize=2)
channel_image = queue.Queue(maxsize=2)
channel_ison = queue.Queue(maxsize=2)
aruco_thread = threading.Thread(target=da.main,args=(channel_pos,channel_image,channel_ison), daemon=True)
aruco_thread.start()
print("Open thymio")
thymio = th.Thymio()
await thymio.iniateLock()
print("Thymio opened")

original_image = channel_image.get()
start = channel_pos.get()
camera_on = channel_ison.get()
path_cm, path, costmap, block_height, block_width, start, goal, display_image, cm_per_pixel =vp.init(original_image,start)
cv2.setMouseCallback("Select  Goal", vp.select_points, original_image)
cm_p_pixel = 77/block_width
path = vp.update(costmap,block_height,block_width,start,vp.points[0],original_image,cm_p_pixel,[])
next_goal = path[0]
current_pos = np.array([start[0],start[1],start[2]])
current_speed = np.array([0,0,0])
camera_position = start
z_predicted = np.concatenate([current_pos,current_speed]) #kalman_state
sigma_predicted = np.diag([0.1,0.1,0.1,0,0,0])



vars_towait = ['prox.horizontal','motor.right.speed','motor.left.speed','motor.right.target','motor.right.target']
vars_thread = threading.Thread(target=thymio.wait, args=(vars_towait,))
vars_thread.start()
time.sleep(1)

while True:
    
    camera_on = channel_ison.get()
    display_image = channel_image.get()
    camera_position = channel_pos.get()
    draw_path(display_image, path_cm)
    cv2.imshow("Select Goal", display_image)
    cv2.waitKey(1)
    
    next_goal = path[0]
    print(f'Next goal : {next_goal}')
    print(f'Current position : {current_pos}')
    thymio.navigate(current_pos, next_goal)
    current_pos = z_predicted[:3] # Is used for the controller
    it_vars = thymio.get_multiple_variables(["motor.left.speed","motor.right.speed"])
    
    z_filtered, sigma_filtered= thymio.filtering_step(z_k_k_1 = z_predicted, 
                                    sigma_k_k_1 = sigma_predicted, 
                                    V_left_measure =it_vars["motor.left.speed"]*thymio.speedConversion, 
                                    V_right_measure = it_vars["motor.right.speed"]*thymio.speedConversion, 
                                    camera_working=camera_on, 
                                    x_measured=camera_position[0], 
                                    y_measured=camera_position[1],
                                    theta_measured= camera_position[2])
    
    z_predicted, sigma_predicted = thymio.prediction_step(z_filtered, sigma_filtered)

    obstacle,detected = thymio.getObstaclePosition()
    if detected:
        print("Obstacle detected")
        #TODO: use the obstacle position to update the costmap
        
        path,costmap = vp.update(costmap,block_height,block_width,current_pos,vp.points[0],display_image,cm_p_pixel,obstacle)
        
    if thymio.robot_close_waypoint(current_pos[:2], next_goal[0], next_goal[1]):
        path.pop(0)
        if len(path) == 0:
            print("Goal reached")
            thymio.set_multiple_variables({"motor.left.target":[0],"motor.right.target":[0]})
            vars_thread.join()
            thymio.stop()
            aruco_thread.join()
            break
        
        
    
    
    
    
    
    
    
    
 







Invalid input! Please enter integers.


TypeError: cannot unpack non-iterable NoneType object